<a href="https://colab.research.google.com/github/thebishorup/nlp/blob/master/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ktrain

  Using cached https://files.pythonhosted.org/packages/f6/91/e2b47f920e48c0b6ba1916000315b4347c4f727063fb0e474a5cde60bf85/ktrain-0.20.2.tar.gz
     |████████████████████████████████| 421.8MB 21kB/s 
     |████████████████████████████████| 983kB 3.4MB/s 
     |████████████████████████████████| 245kB 7.7MB/s 
     |████████████████████████████████| 890kB 8.5MB/s 
     |████████████████████████████████| 471kB 5.5MB/s 
     |████████████████████████████████| 3.9MB 15.3MB/s 
     |████████████████████████████████| 450kB 41.3MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 1.1MB 35.3MB/s 
     |████████████████████████████████| 3.0MB 54.9MB/s 
     |████████████████████████████████| 890kB 43.1MB/s 
  Created wheel for ktrain: filename=ktrain-0.20.2-cp36-none-any.whl size=25268858 sha256=4b64c2d5140c8fa6c1d2aaf24ca0ff279cf0bd0103e13c5c8aa054633f1147b0
  Stored in directory: /root/.cache/pip/wheels/39/cb/df/46a2a4009de6e87d908d89dabdd78216bf73

In [3]:
import tensorflow as tf

In [4]:
tf.__version__

'2.1.0'

In [5]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [6]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

In [7]:
data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx', dtype=str)

In [8]:
data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx', dtype=str)

In [9]:
data_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [10]:
data_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


In [11]:
data_train.shape, data_test.shape

((25000, 2), (25000, 2))

## PRE-PROCESSING

In [12]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=data_train, text_column='Reviews', label_columns='Sentiment', val_df=data_test, maxlen=500,preprocess_mode='bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


## PREPARE MODEL

In [13]:
model = text.text_classifier(name='bert', train_data=(X_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 500
done.


In [16]:
learner = ktrain.get_learner(model=model, train_data=(X_train, y_train), val_data=(X_test, y_test), batch_size=6)

## FINDING BEST LEARNING RATE

In [17]:
# WARNING: THIS PROCESS MAY TAKE DAYS TO COMPLETE
#learner.lr_find()
#learner.lr_gra

In [ ]:
learner.fit_onecycle(lr=2e-5, epochs=1)



begin training using onecycle policy with max lr of 2e-05...
Train on 25000 samples, validate on 25000 samples
24996/25000 [============================>.] - ETA: 1s - loss: 0.2465 - accuracy: 0.8979

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
data = ['This is such a crazy movie', 'This is totally waste of time', 'Not a bad movie. Money well spent']

In [ ]:
predictor.predict(data)

## GETTING PROBABLITY

In [ ]:
predictor.predict(data, return_proba = True)

## SAVIND AND LOADING FINE TUNE MODEL

In [ ]:
predictor.save('/content/bert')

In [ ]:
!zip -r /content/bert.zip /content/bert

## LOADING PREDICTOR FROM SAVED MODEL

In [ ]:
predictor_load = ktrain.load_predictor('/content/bert')

In [ ]:
## GETTING PREDICTOR CLASSES
predictor_load.get_classes()

In [ ]:
predictor_load.predict(data)